In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

Using cpu


In [1]:
d_model = 512
d_hidden = 2048
h = 1
d_k = d_v = d_model/h
max_length = 256

class FeedForward(nn.Module):
    def __init__(self):
        super(FeedForward, self).__init__()
        self.W1 = nn.Linear(d_model, d_hidden)
        self.W2 = nn.Linear(d_hidden, d_model)
        self.ffStack = nn.Sequential(
            self.W1(),
            nn.ReLU(),
            self.W2()
        )
    def forward(self, x):
        return self.ffStack(x)
    

class SelfAttention(nn.Module):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def forward(self, Q, K, V):
        return F.softmax((Q@K.T) * (1/torch.sqrt(d_k)), dim=1) @ V

#single head for now. Will fix later
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.Q_proj = nn.Linear(d_model, d_k*h)
        self.K_proj = nn.Linear(d_model, d_k*h)
        self.V_proj = nn.Linear(d_model, d_v*h)
        self.out_proj = nn.Linear(d_v*h, d_model)
        self.self_attention = SelfAttention()
    
    def forward(self, Q, K, V):
        Q_inp = self.Q_proj(Q)
        K_inp = self.K_proj(K)
        V_inp = self.V_proj(V)
        attn = self.self_attention(Q_inp, K_inp, V_inp)
        return self.out_proj(attn)
    

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.multi_head_attention = MultiHeadAttention()
        self.ffn = FeedForward()
        self.attn_norm = nn.LayerNorm(d_model)
        self.ffn_norm = nn.LayerNorm(d_model)
    
    def forward(self, x):
        x = self.multi_head_attention(x, x, x) + x
        x = self.attn_norm(x)
        x = self.ffn(x) + x
        x = self.ffn_norm(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self):
        super(PositionalEncoding, self).__init__()
        self.p_encode = torch.empty(max_length, d_model)
        for k in range(max_length):
            for i in range(d_model):
                theta = k/10000**(2*i/d_model)
                if i%2 == 0:
                    self.p_encode[k][i] = torch.sin(theta)
                else:
                    self.p_encode[k][i] = torch.cos(theta)
    
    def forward(self, x):
        return self.p_encode + x





NameError: name 'nn' is not defined

In [16]:
tensor1 = torch.randn((3,4))
print(tensor1)
f = nn.LayerNorm(4)
tensor1 = f(tensor1)
print(tensor1)

tensor([[ 1.4547,  2.3426, -0.3314,  0.3432],
        [-1.8603, -2.8984, -0.2083, -0.5121],
        [-1.6965, -0.9645,  0.4901, -0.9244]])
tensor([[ 0.4901,  1.3561, -1.2521, -0.5941],
        [-0.4543, -1.4159,  1.0759,  0.7944],
        [-1.1653, -0.2408,  1.5962, -0.1902]],
       grad_fn=<NativeLayerNormBackward0>)


In [21]:
tensor1 = torch.randn((3,4,5))
tensor2 = torch.randn((3,4,5))
tensor2 = tensor2.transpose(0,1)
print(tensor1.shape)
print(tensor2.shape)
(tensor1@tensor2).shape

torch.Size([3, 4, 5])
torch.Size([4, 3, 5])


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 0